In [1]:
import sys
import math
import ctypes
import pyqbdi
import struct

In [2]:
def insnCB(vm, gpr, fpr, data):
    instAnalysis = vm.getInstAnalysis()
    print("0x{:x}: {}".format(instAnalysis.address, instAnalysis.disassembly))
    return pyqbdi.CONTINUE

In [3]:

# get sin function ptr
if sys.platform == 'darwin':
    libmname = 'libSystem.dylib'
elif sys.platform == 'win32':
    libmname = 'api-ms-win-crt-math-l1-1-0.dll'
else:
    libmname = 'libm.so.6'
libm = ctypes.cdll.LoadLibrary(libmname)
funcPtr = ctypes.cast(libm.sin, ctypes.c_void_p).value

# init VM
vm = pyqbdi.VM()

# create stack
state = vm.getGPRState()
addr = pyqbdi.allocateVirtualStack(state, 0x100000)
assert addr is not None

# instrument library and register memory access
vm.addInstrumentedModuleFromAddr(funcPtr)
vm.recordMemoryAccess(pyqbdi.MEMORY_READ_WRITE)

# add callbacks on instructions
vm.addCodeCB(pyqbdi.PREINST, insnCB, None)

# Cast double arg to long and set FPR
arg = 1.0
carg = struct.pack('<d', arg)
fpr = vm.getFPRState()
fpr.xmm0 = carg

# call sin(1.0)
pyqbdi.simulateCall(state, 0x42424242)
success = vm.run(funcPtr, 0x42424242)

# Retrieve output FPR state
fpr = vm.getFPRState()
# Cast long arg to double
res = struct.unpack('<d', fpr.xmm0[:8])[0]
print("%f (python) vs %f (qbdi)" % (math.sin(arg), res))

# cleanup
pyqbdi.alignedFree(addr)


0x7fce4eecc8f0: 	push	rbx
0x7fce4eecc8f1: 	sub	rsp, 48
0x7fce4eecc8f5: 	mov	rax, qword ptr fs:[40]
0x7fce4eecc8fe: 	mov	qword ptr [rsp + 40], rax
0x7fce4eecc903: 	xor	eax, eax
0x7fce4eecc905: 	vstmxcsr	dword ptr [rsp + 24]
0x7fce4eecc90b: 	mov	edx, dword ptr [rsp + 24]
0x7fce4eecc90f: 	mov	eax, edx
0x7fce4eecc911: 	and	ah, -97
0x7fce4eecc914: 	mov	dword ptr [rsp + 32], eax
0x7fce4eecc918: 	vmovq	rax, xmm0
0x7fce4eecc91d: 	sar	rax, 32
0x7fce4eecc921: 	and	eax, 2147483647
0x7fce4eecc926: 	and	edx, 24576
0x7fce4eecc92c: 	jne	1054
0x7fce4eecc932: 	cmp	eax, 1045430271
0x7fce4eecc937: 	jg	55
0x7fce4eecc970: 	xor	ebx, ebx
0x7fce4eecc972: 	cmp	eax, 1072390143
0x7fce4eecc977: 	jg	267
0x7fce4eecca88: 	cmp	eax, 1073965308
0x7fce4eecca8d: 	jle	381
0x7fce4eeccc10: 	vmovq	xmm3, qword ptr [rip + 81480]
0x7fce4eeccc18: 	vmovsd	xmm1, qword ptr [rip + 99088]
0x7fce4eeccc20: 	lea	rcx, [rip + 233913]
0x7fce4eeccc27: 	vmovsd	xmm5, qword ptr [rip + 99185]
0x7fce4eeccc2f: 	vmovsd	xmm4, qword ptr [rip + 10264